In [34]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import datetime as dt
import re
today_date = dt.date.today().strftime('%Y-%m-%d')
import spacy
from gensim.models import Word2Vec
from collections import defaultdict
from gensim.models import KeyedVectors
from gensim.test.utils import get_tmpfile
from gensim.models.phrases import Phrases, Phraser
import multiprocessing
import time

In [2]:
# query_list = [
#             (f"(TheCircle (#TheCircle) lang:en  since:2022-05-01 until:{today_date})"),
#             (f"(theCircle (#TheCircleNetflix) lang:en  since:2022-05-01 until:{today_date})"),
#             (f"(TheCircle (#TheCircleNetflix) lang:en  since:2022-05-01 until:{today_date})"),
#             (f"(thecircle (#TheCircle) lang:en  since:2022-05-01 until:{today_date})"),
#             (f"(the circle (#TheCircle) lang:en  since:2022-05-01 until:{today_date})"),
#             (f"(the circle (#TheCircleNetflix) lang:en  since:2022-05-01 until:{today_date})") ,
#             (f"(the circle (#TheCircle4) lang:en  since:2022-05-01 until:{today_date})") ,
                     
# ]
# tweets = []
# limit = 50000

# for query in query_list:
#     for tweet in sntwitter.TwitterSearchScraper(query).get_items():
#         if len(tweets) > limit:
#             break
#         else:
#             tweets.append([tweet.date, tweet.user.username, tweet.content])
            
# df = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet'])
# # to save to csv
# df.to_csv('tweet_combo_all.csv', index=False)

In [3]:
import pandas as pd
twt =pd.read_csv('tweet_combo_all.csv')
twt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50001 entries, 0 to 50000
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  50001 non-null  int64 
 1   Date        50001 non-null  object
 2   User        50001 non-null  object
 3   Tweet       50001 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.5+ MB


In [27]:
twt.head()

,Unnamed: 0,Date,User,Tweet
0,0,2022-06-10 23:12:16+00:00,IamJacks5150,#VANHALEN @VanHalen songs we want @RealMadAnth...
1,1,2022-06-10 22:16:02+00:00,kaitlin_xo_,Season 2 of #TheCircle is coming with the dram...
2,2,2022-06-10 20:06:11+00:00,realfscorpio,Season 4 was WIIIIIIIIIIIILD 🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥 #theci...
3,3,2022-06-10 19:55:13+00:00,realfscorpio,Season four and season two had the realest peo...
4,4,2022-06-10 19:49:56+00:00,TheListDaily,Exclusive Interview: #TheCircle's @MichelleBut...


In [5]:
df = twt

## Data preprocessing

In [28]:
#check for nan
df = df.dropna().drop_duplicates().reset_index(drop=True)

TypeError: unhashable type: 'list'

In [24]:
df['tweet_pp'][4]

'exclusive interview thecircle @michellebuteau host duty hit @netflix love recent winner catch episode adulte michelle buteau jordan carlos right URL URL'

In [8]:
# !python -m spacy download en_core_web_sm

In [9]:
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

In [10]:
# function to extract hashtags:
def extract_hashtag(text):
    hashtag_list = []
         # the regular expression
    regex = "#(\w+)"
     
    # extracting the hashtags
    hashtag_list = re.findall(regex, text)
    
    return hashtag_list


In [11]:
df["hashtag"] = [extract_hashtag(text) for text in df["Tweet"]]
df

,Unnamed: 0,Date,User,Tweet,hashtag
0,0,2022-06-10 23:12:16+00:00,IamJacks5150,#VANHALEN @VanHalen songs we want @RealMadAnth...,"[VANHALEN, TheCircle]"
1,1,2022-06-10 22:16:02+00:00,kaitlin_xo_,Season 2 of #TheCircle is coming with the dram...,[TheCircle]
2,2,2022-06-10 20:06:11+00:00,realfscorpio,Season 4 was WIIIIIIIIIIIILD 🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥 #theci...,[thecircle]
3,3,2022-06-10 19:55:13+00:00,realfscorpio,Season four and season two had the realest peo...,[thecircle]
4,4,2022-06-10 19:49:56+00:00,TheListDaily,Exclusive Interview: #TheCircle's @MichelleBut...,[TheCircle]
...,...,...,...,...,...
49996,49996,2022-05-13 20:36:47+00:00,MikeGiuda,this is a Yu Ling stan account #TheCircle #The...,"[TheCircle, TheCircleNetflix]"
49997,49997,2022-05-13 20:31:26+00:00,ZahraFatih72,@GregDav94758992 The AUDACITY of Channel 4 sho...,"[TheCircle, Gogglebox]"
49998,49998,2022-05-13 20:29:12+00:00,TamiOsama,I honestly love frank from #TheCircle season 4...,[TheCircle]
49999,49999,2022-05-13 20:25:11+00:00,elliot_gonzalez,Good to see #TheCircle back on Channel 4… sham...,"[TheCircle, Gogglebox]"


In [13]:
# function to perform  preprocessing on the tweets

def spacy_clean_text( doc, min_token_len=2,
    irrelevant_pos=["ADV", "PRON", "CCONJ", "PUNCT", "PART", "DET", "ADP", "SPACE"]):
    """function to perform  preprocessing on the tweets and split them into words"""
    
    clean_text = []
    for token in doc:
        # if token.like_email:
        #     clean_text.append("twt_user")
        if token.like_url:
            clean_text.append("URL")
        elif (
                token.is_stop == False  # Check if it's not a stopword
                and len(token) > min_token_len  # Check if the word meets minimum threshold
                and token.pos_ not in irrelevant_pos  # Check if the POS is in the acceptable POS tags
                and token.is_punct == False # Check if there is no puctuation
        ):
            lemma = token.lemma_  # Take the lemma of the word
            clean_text.append(lemma.lower())
    return " ".join(clean_text)

In [ ]:
# df1 = df[:3]
# df1

In [14]:
df["tweet_pp"] = [spacy_clean_text(text) for text in nlp.pipe(df["Tweet"])]


In [25]:
df.head()

,Unnamed: 0,Date,User,Tweet,hashtag,tweet_pp
0,0,2022-06-10 23:12:16+00:00,IamJacks5150,#VANHALEN @VanHalen songs we want @RealMadAnth...,"[VANHALEN, TheCircle]",vanhalen @vanhalen song want @realmadanthony s...
1,1,2022-06-10 22:16:02+00:00,kaitlin_xo_,Season 2 of #TheCircle is coming with the dram...,[TheCircle],season thecircle come drama need ep
2,2,2022-06-10 20:06:11+00:00,realfscorpio,Season 4 was WIIIIIIIIIIIILD 🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥 #theci...,[thecircle],season wiiiiiiiiiiiild thecircle
3,3,2022-06-10 19:55:13+00:00,realfscorpio,Season four and season two had the realest peo...,[thecircle],season season real people deserve winner love ...
4,4,2022-06-10 19:49:56+00:00,TheListDaily,Exclusive Interview: #TheCircle's @MichelleBut...,[TheCircle],exclusive interview thecircle @michellebuteau ...


In [17]:
df.to_csv('processed.csv', index=False)

In [ ]:
# text = sub(r"[^A-Za-z0-9^,!?.\/'+]", " ", text)
#     text = sub(r"\+", " plus ", text)
#     text = sub(r",", " ", text)
#     text = sub(r"\.", " ", text)
#     text = sub(r"!", " ! ", text)
#     text = sub(r"\?", " ? ", text)
#     text = sub(r"'", " ", text)
#     text = sub(r":", " : ", text)
#     text = sub(r"\s{2,}", " ", text)

In [18]:
processed_df = pd.read_csv('processed.csv')

In [26]:
processed_df.reset_index(drop=True)

,Unnamed: 0.1,Unnamed: 0,Date,User,Tweet,hashtag,tweet_pp
0,0,0,2022-06-10 23:12:16+00:00,IamJacks5150,#VANHALEN @VanHalen songs we want @RealMadAnth...,"['VANHALEN', 'TheCircle']",vanhalen @vanhalen song want @realmadanthony s...
1,1,1,2022-06-10 22:16:02+00:00,kaitlin_xo_,Season 2 of #TheCircle is coming with the dram...,['TheCircle'],season thecircle come drama need ep
2,2,2,2022-06-10 20:06:11+00:00,realfscorpio,Season 4 was WIIIIIIIIIIIILD 🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥 #theci...,['thecircle'],season wiiiiiiiiiiiild thecircle
3,3,3,2022-06-10 19:55:13+00:00,realfscorpio,Season four and season two had the realest peo...,['thecircle'],season season real people deserve winner love ...
4,4,4,2022-06-10 19:49:56+00:00,TheListDaily,Exclusive Interview: #TheCircle's @MichelleBut...,['TheCircle'],exclusive interview thecircle @michellebuteau ...
...,...,...,...,...,...,...,...
49996,49996,49996,2022-05-13 20:36:47+00:00,MikeGiuda,this is a Yu Ling stan account #TheCircle #The...,"['TheCircle', 'TheCircleNetflix']",ling stan account thecircle thecirclenetflix URL
49997,49997,49997,2022-05-13 20:31:26+00:00,ZahraFatih72,@GregDav94758992 The AUDACITY of Channel 4 sho...,"['TheCircle', 'Gogglebox']",audacity channel show thecircle usa gogglebox ...
49998,49998,49998,2022-05-13 20:29:12+00:00,TamiOsama,I honestly love frank from #TheCircle season 4...,['TheCircle'],love frank thecircle season realist shubham se...
49999,49999,49999,2022-05-13 20:25:11+00:00,elliot_gonzalez,Good to see #TheCircle back on Channel 4… sham...,"['TheCircle', 'Gogglebox']",good thecircle channel shame clip brilliant ne...


In [29]:
sent = [row for row in processed_df.tweet_pp]
phrases = Phrases(sent, min_count=1, progress_per=50000)
bigram = Phraser(phrases)
sentences = bigram[sent]
sentences[1]

'season thecircle come drama need ep'

In [36]:
w2v_model = Word2Vec(min_count=3,
                     window=4,
                     vector_size=300,
                     sample=1e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=multiprocessing.cpu_count()-1)

# start = time()

w2v_model.build_vocab(sentences, progress_per=50000)

# print('Time to build vocab: {} mins'.format(round((time() - start) / 60, 2)))

In [37]:
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

# print('Time to train the model: {} mins'.format(round((time() - start) / 60, 2)))

w2v_model.init_sims(replace=True)

C:\Users\17783\AppData\Local\Temp\ipykernel_33540\1548044844.py:5: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v_model.init_sims(replace=True)


In [38]:
w2v_model.save("word2vec.model")

In [41]:
file_export = processed_df.copy()
file_export['tweet_pp_old'] = file_export.tweet_pp
file_export.tweet_pp_old = file_export.tweet_pp_old.str.join(' ')
file_export.tweet_pp = file_export.tweet_pp.apply(lambda x: ' '.join(bigram[x]))

In [42]:
file_export

,Unnamed: 0.1,Unnamed: 0,Date,User,Tweet,hashtag,tweet_pp,tweet_pp_old
0,0,0,2022-06-10 23:12:16+00:00,IamJacks5150,#VANHALEN @VanHalen songs we want @RealMadAnth...,"['VANHALEN', 'TheCircle']",v a n h a l e n @ v a n h a l e n s o n g ...,v a n h a l e n @ v a n h a l e n s o n g ...
1,1,1,2022-06-10 22:16:02+00:00,kaitlin_xo_,Season 2 of #TheCircle is coming with the dram...,['TheCircle'],s e a s o n t h e c i r c l e c o m e d ...,s e a s o n t h e c i r c l e c o m e d ...
2,2,2,2022-06-10 20:06:11+00:00,realfscorpio,Season 4 was WIIIIIIIIIIIILD 🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥 #theci...,['thecircle'],s e a s o n w i i i i i i i i i i i i l d ...,s e a s o n w i i i i i i i i i i i i l d ...
3,3,3,2022-06-10 19:55:13+00:00,realfscorpio,Season four and season two had the realest peo...,['thecircle'],s e a s o n s e a s o n r e a l p e o p ...,s e a s o n s e a s o n r e a l p e o p ...
4,4,4,2022-06-10 19:49:56+00:00,TheListDaily,Exclusive Interview: #TheCircle's @MichelleBut...,['TheCircle'],e x c l u s i v e i n t e r v i e w t h e ...,e x c l u s i v e i n t e r v i e w t h e ...
...,...,...,...,...,...,...,...,...
49996,49996,49996,2022-05-13 20:36:47+00:00,MikeGiuda,this is a Yu Ling stan account #TheCircle #The...,"['TheCircle', 'TheCircleNetflix']",l i n g s t a n a c c o u n t t h e c i ...,l i n g s t a n a c c o u n t t h e c i ...
49997,49997,49997,2022-05-13 20:31:26+00:00,ZahraFatih72,@GregDav94758992 The AUDACITY of Channel 4 sho...,"['TheCircle', 'Gogglebox']",a u d a c i t y c h a n n e l s h o w t ...,a u d a c i t y c h a n n e l s h o w t ...
49998,49998,49998,2022-05-13 20:29:12+00:00,TamiOsama,I honestly love frank from #TheCircle season 4...,['TheCircle'],l o v e f r a n k t h e c i r c l e s e ...,l o v e f r a n k t h e c i r c l e s e ...
49999,49999,49999,2022-05-13 20:25:11+00:00,elliot_gonzalez,Good to see #TheCircle back on Channel 4… sham...,"['TheCircle', 'Gogglebox']",g o o d t h e c i r c l e c h a n n e l ...,g o o d t h e c i r c l e c h a n n e l ...


In [43]:
file_export[['Date', 'User', 'Tweet','hashtag', 'tweet_pp']].to_csv('cleaned_dataset.csv', index=False)

In [44]:
## Applying K-Means clustering to the model

In [46]:
from sklearn.cluster import KMeans

In [47]:
#load the word2vec model

In [48]:
word_vectors = Word2Vec.load("word2vec.model").wv

In [49]:
model = KMeans(n_clusters=3, max_iter=1000, random_state=True, n_init=50).fit(X=word_vectors.vectors.astype('double'))

In [55]:
word_vectors.similar_by_vector(model.cluster_centers_[2], topn=10, restrict_vocab=None)

[('é', 0.9533335566520691),
 ('a', 0.933819055557251),
 (' ', 0.9315211772918701),
 ('v', 0.9306424856185913),
 ('o', 0.9254785776138306),
 ('t', 0.923130989074707),
 ('l', 0.9186508059501648),
 ('d', 0.9179863929748535),
 ('e', 0.912469208240509),
 ('g', 0.9114618301391602)]

In [51]:
positive_cluster_center = model.cluster_centers_[0]
negative_cluster_center = model.cluster_centers_[1]
neutral_cluster_center = model.cluster_centers_[2]

In [52]:
model.cluster_centers_

array([[-3.41355850e-02,  9.20724403e-02, -5.97087325e-03,
         4.20054542e-02, -9.60583840e-03, -1.04669539e-01,
         5.33134418e-02,  1.39430159e-01,  5.03956021e-02,
        -6.33171335e-02,  8.43114036e-02, -3.05300843e-02,
         1.95741073e-02,  5.38932247e-02, -5.68422760e-02,
        -1.01012322e-02,  7.96049871e-02, -3.86735607e-02,
        -1.34602531e-02, -5.05481362e-02,  1.16005045e-02,
         3.57634141e-02,  5.30181568e-02,  3.89608403e-02,
         2.37898714e-03,  6.69185916e-03, -5.40041721e-02,
         7.80867077e-02, -6.20862695e-02, -4.46502398e-02,
         1.29737243e-02, -6.53747618e-03,  2.97062389e-02,
         5.56835227e-04, -2.71607019e-02,  9.96616433e-02,
         7.88136672e-02, -4.90215424e-02, -3.66502422e-02,
        -1.50300657e-02, -8.43179759e-02, -8.69769555e-03,
        -4.69621848e-02, -4.83326392e-02,  6.19537066e-02,
         3.92481056e-02, -1.02045216e-02, -4.92010640e-03,
         2.12302899e-02,  1.13183528e-02,  1.61991730e-0